In [1]:
"""
# Python script for post processing the weights of the stroopwafel sampled data. 
This is the script that can be implemented in the postProcessing.py of COMPAS.
**This jupyter notebook is part 3 of implementing adaptive importance sampling in COMPAS ** <br>
See Broekgaarden+18 and the AdaptiveImportanceSampling folder on gitlab. 
Last updated on 08-09-2018; Floor Broekgaarden with help of Simon Stevenson
***
"""

import numpy as np
import scipy
from scipy.stats import multivariate_normal
from scipy import integrate
import math
import random
import os

import h5py as h5 





Rcoeff =\
    np.asarray([[1.71535900,    0.62246212,     -0.92557761,    -1.16996966,    -0.30631491],\
    [6.59778800,    -0.42450044,    -12.13339427,   -10.73509484,   -2.51487077],\
    [10.08855000,   -7.11727086,    -31.67119479,   -24.24848322,   -5.33608972],\
    [1.01249500,    0.32699690,     -0.00923418,    -0.03876858,    -0.00412750],\
    [0.07490166,    0.02410413,     0.07233664,     0.03040467,     0.00197741],\
    [0.01077422,    0.00000000,     0.00000000,     0.00000000,     0.00000000],\
    [3.08223400,    0.94472050,     -2.15200882,    -2.49219496,    -0.63848738],\
    [17.84778000,   -7.45345690,    -48.96066856,   -40.05386135,   -9.09331816],\
    [0.00022582,    -0.00186899,    0.00388783,     0.00142402,     -0.00007671]])

    
RsolToAU = 0.00465047 #    # // Solar radius (in AU)

                
def calculateRadiusCoefficient(whichCoefficient, logMetallicityXi):
    """
    /*
     Calculate an alpha-like metallicity dependent constant for the radius
     
     Equation 4 in Tout et al 1996
     
     Parameters
     -----------
     whichCoefficient : int
     Which coefficient to evaluate:
     COEFF_THETA     = 0
     COEFF_IOTA      = 1
     COEFF_KAPPA     = 2
     COEFF_LAMBDA    = 3
     COEFF_MU        = 4
     COEFF_NU        = 5
     COEFF_XI        = 6
     COEFF_OMICRON   = 7
     COEFF_PI        = 8
     
     logMetallicityXi:
     log10(Metallicity / Zsol) where Metallicity = Z (Z = 0.02 = Zsol)
     
     Returns
     ----------
     alpha : float
     Constant alpha

     */
    """
    
    a = Rcoeff[whichCoefficient][0]
    b = Rcoeff[whichCoefficient][1]
    c = Rcoeff[whichCoefficient][2]
    d = Rcoeff[whichCoefficient][3]
    e = Rcoeff[whichCoefficient][4]

  
    return a + b * logMetallicityXi + c * np.power(logMetallicityXi, 2.0) + d * np.power(logMetallicityXi, 3.0) + e * np.power(logMetallicityXi, 4.0)



#// Calculate metallicity dependent radius coefficients
def    calculateAllRadiusCoefficients(logMetallicityXi):

    """
    Calculate metallicity dependent radius coefficients at start up.

    See calculateRadiusCoefficient

    Parameters
    -----------
    logMetallicityXi : double
        log of metallicity

    Returns
    --------
    """
    
    COEFF_THETA     = 0
    COEFF_IOTA      = 1
    COEFF_KAPPA     = 2
    COEFF_LAMBDA    = 3
    COEFF_MU        = 4
    COEFF_NU        = 5
    COEFF_XI        = 6
    COEFF_OMICRON   = 7
    COEFF_PI        = 8    
    
    
    radius_coefficient_theta   = calculateRadiusCoefficient(COEFF_THETA,   logMetallicityXi);
    radius_coefficient_iota    = calculateRadiusCoefficient(COEFF_IOTA,    logMetallicityXi);
    radius_coefficient_kappa   = calculateRadiusCoefficient(COEFF_KAPPA,   logMetallicityXi);
    radius_coefficient_lamda   = calculateRadiusCoefficient(COEFF_LAMBDA,  logMetallicityXi);
    radius_coefficient_mu      = calculateRadiusCoefficient(COEFF_MU,      logMetallicityXi);
    radius_coefficient_nu      = calculateRadiusCoefficient(COEFF_NU,      logMetallicityXi);
    radius_coefficient_xi      = calculateRadiusCoefficient(COEFF_XI,      logMetallicityXi);
    radius_coefficient_omicron = calculateRadiusCoefficient(COEFF_OMICRON, logMetallicityXi);
    radius_coefficient_Pi      = calculateRadiusCoefficient(COEFF_PI,      logMetallicityXi);    
    
    
    

    return radius_coefficient_theta, \
                radius_coefficient_iota, radius_coefficient_kappa,\
                radius_coefficient_lamda, radius_coefficient_mu, \
                radius_coefficient_nu, radius_coefficient_xi, \
                radius_coefficient_omicron, radius_coefficient_Pi
                


   


##########################################


def inverse_sampling_from_power_law(Nprior, power, xmax, xmin):
    # // This function draws samples from a power law distribution p(x) ~ x^(n) between xmin and xmax
    # // Checked against python code for the same function
    
    u = np.random.rand(Nprior)              #// Draw N random number between 0 and 1
    
    if(power == -1.0):
        return np.exp(u*np.log(xmax/xmin))*xmin
    
    else:
        return np.power((u*(np.power(xmax, power+1.0) - np.power(xmin, power+1.0)) + np.power(xmin, power+1.0)), 1.0/(power+1.0))



def sampling_from_e(Nprior, eccentricity_distribution, eMax, eMin):


    if eccentricity_distribution=='ZERO':
        m_Eccentricity =   np.zeros(Nprior) 

    elif(eccentricityDistribution=="FLAT"):
        ePower = 0
        m_Eccentricity = inverse_sampling_from_power_law(Nprior, ePower, eMax, eMin)
    
    elif((eccentricityDistribution=="THERMALISED") or (eccentricityDistribution=="THERMAL")):
        # Thermal eccentricity distribution p(e) = 2e
        ePower = 1
        m_Eccentricity = inverse_sampling_from_power_law(Nprior, ePower, eMax, eMin)
    else:
        print('Error: your chosen eccentricityDistribution, ',eccentricity_distribution, ' ,does not exist, =')

    return m_Eccentricity



def  RadiusZAMS(MZAMS, radius_coefficient_theta, \
                radius_coefficient_iota, radius_coefficient_kappa,\
                radius_coefficient_lamda, radius_coefficient_mu, \
                radius_coefficient_nu, radius_coefficient_xi, \
                radius_coefficient_omicron, radius_coefficient_Pi):
    """Calculate Zero Age Main Sequence (ZAMS) Radius  using Pols et al formulae
        Calculate radius at ZAMS in units of Rsol

         Equation 2 in Tout et al 1996

         Parameters
         -----------
         MZAMS : float
            Zero age main sequence mass in Msol

         Returns
         --------
         RZAMS : float
         Radius in units of Rsol
     
     """



    top     = radius_coefficient_theta * np.power(MZAMS,(2.5)) +\
    radius_coefficient_iota * np.power(MZAMS,(6.5)) + \
    radius_coefficient_kappa * np.power(MZAMS,(11.0)) + \
    radius_coefficient_lamda * np.power(MZAMS,(19.0)) +\
    radius_coefficient_mu * np.power(MZAMS,(19.5))

    bottom  = radius_coefficient_nu + radius_coefficient_xi * np.power(MZAMS,(2.0)) + radius_coefficient_omicron * np.power(MZAMS,(8.5)) + np.power(MZAMS,(18.5)) + radius_coefficient_Pi * np.power(MZAMS,(19.5))
    
    RadiusZAMS = top/bottom 
    
    return RadiusZAMS





def rocheLobeRadius(Mass_i, Mass_j):
    """
     Calculate the roche lobe radius using the chosen approximation
     
     By default uses the fit given in Equation 2 of Eggleton 1983 (Could in principle pass it options and use other fits like Sepinsky+)
     
     Parameters
     -----------
     Mass_i : double
        Primary mass
     Mass_j : double
        Secondary mass
     
     Returns
     --------
     R_L / a : double
     Radius of Roche lobe in units of the semi-major axis a

     """

    q = Mass_i/Mass_j
    top = 0.49;
    bottom = 0.6 + np.power(q, -2.0/3.0) * np.log(1.0 + np.power(q, 1.0/3.0))

    return top/bottom;




def CalculateFractionRejectedPriors(eccentricity_distribution, eMax, eMin, minimum_secondary_mass):
    """Calcultes the fraction of samples that falls outside of the parameter space, 
    because of rejection sampling by either RLOFonZAMS or M2 < minMass in COMPAS
    """

    # monte carlo simulation with resolution 1E8 samples
    sum_rejected, NPrior = 0, int(1E8) 

    samples_M1 = sampling_from_IMF(NPrior)  
    samples_a = sampling_from_a(NPrior)
    samples_q = sampling_from_q(NPrior) 

    
    
    ########## CALCULATE MASK RLOFonZAMS DETAILED:########################
    samples_M2 = samples_q*samples_M1
    m_R1ZAMS             = RadiusZAMS(samples_M1, radius_coefficient_theta, radius_coefficient_iota, \
                                     radius_coefficient_kappa, radius_coefficient_lamda, \
                                     radius_coefficient_mu, radius_coefficient_nu, \
                                     radius_coefficient_xi, radius_coefficient_omicron,\
                                     radius_coefficient_Pi)
    m_R2ZAMS             = RadiusZAMS(samples_M2, radius_coefficient_theta, radius_coefficient_iota, \
                                     radius_coefficient_kappa, radius_coefficient_lamda, \
                                     radius_coefficient_mu, radius_coefficient_nu, \
                                     radius_coefficient_xi, radius_coefficient_omicron,\
                                     radius_coefficient_Pi)


    m_Eccentricity = sampling_from_e(NPrior, eccentricity_distribution, eMax, eMin)
    m_SemiMajorAxis=   samples_a # sampled MC 

    m_RadiusRL1             = rocheLobeRadius(samples_M1, samples_M2);
    m_RadiusRL2             = rocheLobeRadius(samples_M2, samples_M1);


    rocheLobeTracker1 = (m_R1ZAMS*RsolToAU)/(m_SemiMajorAxis*(1.0 - m_Eccentricity)*m_RadiusRL1)
    rocheLobeTracker2 = (m_R2ZAMS*RsolToAU)/(m_SemiMajorAxis*(1.0 - m_Eccentricity)*m_RadiusRL2)


    
    maskRLOFonZAMS1 = (rocheLobeTracker1 > 1.0) 
    maskRLOFonZAMS2 =   (rocheLobeTracker2 > 1.0)
    maskRLOFonZAMS = (rocheLobeTracker1 > 1.0) |  (rocheLobeTracker2 > 1.0)


    
    # mask samples that are inside parameter space     
    # m2 with too low masses are rejected within COMPAS
    # approximate property to not have RLOFonZAMS
    # mask samples that we keep
    mask = (((samples_q*samples_M1)>= minimum_secondary_mass) \
    & (maskRLOFonZAMS==0) )

    sum_rejected = len(mask) - np.sum(mask)   # total rejected samples of all gaussians


    # calculate the fraction of samples that is rejected
    Prej = sum_rejected / (NPrior)


    return Prej




def CalculateFractionRejectedGaussians(masterDirectory, eccentricity_distribution, eMax, eMin, minimum_secondary_mass): #, minSeparationRLOFonZAMS):
    """Calcultes the fraction of samples that falls outside of the parameter space, F_rej
    when sampling from the Gaussians q(x) (see also Eq. 8 and 9 in https://arxiv.org/pdf/1905.00910.pdf)

    """


    d_gaussians = np.genfromtxt(masterDirectory + '/STROOPWAFELgaussians.txt', skip_header=2)
    targetmeans = d_gaussians[:,0:3]
    targetstd   = d_gaussians[:,3: ]

    sum_rejected, NperGauss = 0, 10000

    std_m1, std_a, std_q =  np.transpose(targetstd) # obtain std (\sigma) from AISgaussians.txt
    mu_m1, mu_a, mu_q = np.transpose(targetmeans)

    # for all Gaussians count how many samples fall outside of parameter space
    for i in range(len(targetmeans)):
        # sample NperGauss samples from i-th Gaussian for M_1, a and q. To the power 2 to obtain sigma^2 instead of std (sigma)
        samples_M1 = (multivariate_normal.rvs(mean = mu_m1[i], cov = std_m1[i]**2, size = NperGauss))  
        samples_a = 10**(multivariate_normal.rvs(mean = mu_a[i], cov = std_a[i]**2, size = NperGauss)) 
        samples_q = multivariate_normal.rvs(mean = mu_q[i], cov = std_q[i]**2, size = NperGauss)  

        # mask samples that are inside parameter space  
       # m2 with too low masses are rejected within COMPAS
        # approximate property to not have RLOFonZAMS
        

        ########## CALCULATE MASK RLOFonZAMS:########################
        samples_M2 = samples_q*samples_M1
        m_R1ZAMS             = RadiusZAMS(samples_M1, radius_coefficient_theta, radius_coefficient_iota, \
                                         radius_coefficient_kappa, radius_coefficient_lamda, \
                                         radius_coefficient_mu, radius_coefficient_nu, \
                                         radius_coefficient_xi, radius_coefficient_omicron,\
                                         radius_coefficient_Pi)
        m_R2ZAMS             = RadiusZAMS(samples_M2, radius_coefficient_theta, radius_coefficient_iota, \
                                         radius_coefficient_kappa, radius_coefficient_lamda, \
                                         radius_coefficient_mu, radius_coefficient_nu, \
                                         radius_coefficient_xi, radius_coefficient_omicron,\
                                         radius_coefficient_Pi)


        m_Eccentricity =   sampling_from_e(NperGauss, eccentricity_distribution, eMax, eMin)
        m_SemiMajorAxis=   samples_a # sampled MC 

        m_RadiusRL1             = rocheLobeRadius(samples_M1, samples_M2);
        m_RadiusRL2             = rocheLobeRadius(samples_M2, samples_M1);


        rocheLobeTracker1 = (m_R1ZAMS*RsolToAU)/(m_SemiMajorAxis*(1.0 - m_Eccentricity)*m_RadiusRL1)
        rocheLobeTracker2 = (m_R2ZAMS*RsolToAU)/(m_SemiMajorAxis*(1.0 - m_Eccentricity)*m_RadiusRL2)

        maskRLOFonZAMS = (rocheLobeTracker1 > 1.0) |  (rocheLobeTracker2 > 1.0)
        
       
        mask = ((samples_M1 >= initial_mass_min) & (samples_M1 <= initial_mass_max) & \
         (samples_a >= (10.**semi_major_axis_min)) &  (samples_a <= (10.**semi_major_axis_max)) \
                    &  (samples_q >= mass_ratio_min) & (samples_q <= mass_ratio_max) \
                    & ((samples_q*samples_M1)>= minimum_secondary_mass) \
                    & (maskRLOFonZAMS == 0 ) )
                
                
        Nrejected = len(mask) - np.sum(mask) 
        sum_rejected += Nrejected  # total rejected samples of all gaussians

    # calculate the nr of samples total drawn from Gaussians
    NsamplesFromGaussians = NperGauss * len(targetmeans) 
    # calculate the fraction of samples that is rejected
    fraction_rejected = sum_rejected / (NsamplesFromGaussians)

    del samples_M1
    del samples_a
    del samples_q
    del samples_M2
    del mask
    # del sum_rejected
    del rocheLobeTracker2
    del rocheLobeTracker1
    del maskRLOFonZAMS
    del m_Eccentricity
    del m_RadiusRL2
    del m_RadiusRL1
    del m_SemiMajorAxis
    del m_R1ZAMS
    del m_R2ZAMS
    del Nrejected
    del NsamplesFromGaussians
    del sum_rejected
    del targetmeans
    del targetstd
    del d_gaussians
    del std_q
    del std_a
    del std_m1

    return fraction_rejected



def calculatessmallq(array, masterDirectory):  #new version
    '''function that calculates the pdf of the instrumental distribution
    input is an array of samples, array of AISmeans and AIScov, 
    it returns the probability for each of the samples for the mixture pdf
    this is Equation 8 (q(x)) of https://arxiv.org/pdf/1905.00910.pdf
    '''
    
    
    d_gaussians = np.genfromtxt(masterDirectory + '/STROOPWAFELgaussians.txt', skip_header=2)
    targetmeans, targetstd = d_gaussians[:,0:3], d_gaussians[:,3: ]
    Ngaussians = len(targetmeans)

    # make a matrix which we will fill up with contributions to the PDF from each Gaussian
    xPDF = np.zeros((Ngaussians, len(array)))
    
    
    # calculate for each gaussian the probability of the array \
    # then sum all the probabilities together and normalize
    for i in range(Ngaussians):
        COV_M1, COV_a, COV_q =  targetstd[i]**2 # this is \sigma^2 in definition \Sigma, we need to take **2 since the Gaussians feed to COMPAS in c++ are np.sqrt(factor)*cov (so cov instead of cov^2)
        COV = [[COV_M1, 0, 0], [0, COV_a, 0], [0, 0, COV_q]]
        # print len(array), len(targetmeans[i]), len(COV)
        # print 'array', array
        # print 'mean', targetmeans[i]
        # print 'cov' , COV
        xPDF[i, :] = (multivariate_normal.pdf(list(array), targetmeans[i], COV))
    


    qPDF = np.sum(xPDF, axis = 0)  * (float(Ngaussians))**-1  # Eq. 8 of https://arxiv.org/pdf/1905.00910.pdf
    
    del xPDF
    del COV_q
    del COV_a
    del COV_M1
    del COV
    del Ngaussians
    del targetmeans
    del d_gaussians


    return qPDF




def calculatesPi(array):  
    ''' returns the probability for each of the samples for the priors
    see Eq. 1 of https://arxiv.org/pdf/1905.00910.pdf 
    '''
        
    # pdf contribution from prior sampling
    priorM1 =  prior_M1(np.transpose(array)[0])
    priora =  prior_loga(np.transpose(array)[1])
    priorq =  prior_q(np.transpose(array)[2])
    
    piPDF= priorM1 * priora * priorq 
    
    del priorq
    del priorM1
    del priora

    return piPDF


def calculateMixtureWeights(pi_x, q_x,  PIrej, Qrej, fexpl):
    """  """
    #normalization factors for pi(x) and q(x) due to rejection sampling COMPAS
    PInorm =  1./(1.-PIrej) 
    Qnorm =  1./(1.-Qrej) 
    
    numerator =  PInorm * pi_x
    denomenator = (fexpl * PInorm * pi_x)   +  ( (1.-fexpl) * Qnorm * q_x) 
    
    mixtureweights = numerator / denomenator
    
    del numerator
    del denomenator
    del PInorm
    del Qnorm


    return mixtureweights





##############################################
# below functions are needed for rejection sampling normalization RLOFonZAMS
# all functions come from COMPAS .cpp files, and are in COMPAS used to determine whether a binary has RLOFonZAMS



logMetallicityXi = np.log10(0.0001) 
radius_coefficient_theta, \
    radius_coefficient_iota, radius_coefficient_kappa, \
    radius_coefficient_lamda, radius_coefficient_mu, \
    radius_coefficient_nu, radius_coefficient_xi, \
    radius_coefficient_omicron, radius_coefficient_Pi  \
    = calculateAllRadiusCoefficients(logMetallicityXi)



In [11]:
def radiusHertzsprungGap(Mass,  coreMass,  RZAMS,  tau,  logMetallicityXi):
    
    """
     Calculate the radius on the Hertzsprung Gap (HG)
     
     Given by Equation 27 in Hurley et al 2000
     
     Parameters
     -----------
     an_coefficients :  array
     Array containing metallicity dependent an coefficients
     bn_coefficients :  array
     Array containing metallicity dependent bn coefficients
     massCutoffs :  array
     Array containing metallicity dependent mass cutoffs
     Mass : def
     Mass in Msol
     coreMass : def
     Core mass in Msol
     RZAMS : def
     Zero Age Main Sequence (ZAMS) Radius
     tau : def
     Relative time [0,1]
     
     Returns
     --------
     RHG : def
     Radius whilst on the Hertzsprung Gap
     
     
     """
    
    RTMS = radiusEndMainSequence(an_coefficients, Mass, RZAMS, m_randomSeed)
    REHG = radiusEndHertzsprungGap(an_coefficients, bn_coefficients, Mass, coreMass, massCutoffs, logMetallicityXi, m_randomSeed);
    
    return RTMS * np.power((REHG/RTMS), (tau));
    


In [12]:
samples_M1 = np.linspace(5,150, 100)



# mask samples that are inside parameter space  
# m2 with too low masses are rejected within COMPAS
# approximate property to not have RLOFonZAMS


########## CALCULATE MASK RLOFonZAMS:########################

m_R1ZAMS             = RadiusZAMS(samples_M1, radius_coefficient_theta, radius_coefficient_iota, \
                                 radius_coefficient_kappa, radius_coefficient_lamda, \
                                 radius_coefficient_mu, radius_coefficient_nu, \
                                 radius_coefficient_xi, radius_coefficient_omicron,\
                                 radius_coefficient_Pi)



m_R1HG               = radiusHertzsprungGap(Mass=samples_M1,  coreMass=0.9*samples_M1,  RZAMS=m_R1ZAMS,   logMetallicityXi=1)


In [9]:


def radiusEndMainSequence( an_coefficients[nrowsa],  Mass,  RZAMS,  m_randomSeed):
    """
     Calculate the radius at the end of the Main Sequence (MS), R_TMS
     
     Equation 9a, 9b in Hurley et al 2000
     
     Parameters
     -----------
     an_coefficients :  array
     Array of length nrowsa containing the metallicity dependent a_n coefficients for this star
     Mass : def
     Mass in Msol
     RZAMS : def
     Zero Age Main Sequence (ZAMS) radius in Rsol
     
     Returns
     --------
     R_TMS : def
     Radius of a star at the end of the Main Sequence (MS)
     
     
     """
    
    
    # Declare variables
    Masterisk    = an_coefficients[17 - 1] + 0.1;
    RTMS         = 0;                                    # Value we want to calculate and return.
    
    if(Mass <= an_coefficients[17 - 1]):
        top      = an_coefficients[18 - 1] + an_coefficients[19 - 1] * pow(Mass, an_coefficients[21 - 1]);
        bottom   = an_coefficients[20 - 1] + pow(Mass, an_coefficients[22 - 1]);
        RTMS = top / bottom;
        if(Mass < 0.5):
            return std::max(RTMS, 1.5 * RZAMS);
        
        else:
            return RTMS;
        
    
    elif(Mass >= Masterisk):
        top      = cCoefficients[1 - 1] * pow(Mass, 3.0) + an_coefficients[23 - 1] * pow(Mass, an_coefficients[26 - 1]) + an_coefficients[24 - 1] * pow(Mass, (an_coefficients[26 - 1] + 1.5));
        bottom   = an_coefficients[25 - 1] + pow(Mass, 5.0);
        RTMS =  top / bottom;
        return RTMS;
    
    else:
        # For stars with masses between a17, a17 + 0.1 interpolate between the end points (y = mx + c)
        y2_top           = cCoefficients[1 - 1] * pow(Masterisk, 3.0) + an_coefficients[23 - 1] * pow(Masterisk, an_coefficients[26 - 1]) + an_coefficients[24 - 1] * pow(Masterisk, (an_coefficients[26 - 1] + 1.5));
        y2_bottom        = an_coefficients[25 - 1] + pow(Masterisk, 5.0);
        y2               = y2_top / y2_bottom;                       # RTMS(Masterisk)
        
        y1_top           = an_coefficients[18 - 1] + an_coefficients[19 - 1] * pow(an_coefficients[17 - 1], an_coefficients[21 - 1]);
        y1_bottom        = an_coefficients[20 - 1] + pow(an_coefficients[17 - 1], an_coefficients[22 - 1]);
        y1               = y1_top / y1_bottom;                       # RTMS(a17)
        
        gradient_top     = y2 - y1;                                      # y2 - y1 = RTMS(Masterisk) - RTMS(a17)
        gradient_bottom  = 0.1;                                          # x2 - x1 = a17 + 0.1 - a17
        gradient         = gradient_top / gradient_bottom;
        
        intercept        = y1 - gradient * an_coefficients[17 - 1];      # c = y - mx
        return gradient*Mass + intercept;





def radiusEndHertzsprungGap( an_coefficients[nrowsa],  bn_coefficients[nrowsb],  Mass,  coreMass,  massCutoffs[nMassCutoffs],  logMetallicityXi,  m_randomSeed):
    """
     Calculate the radius of a star at the end of the Hertzsprung Gap R_EHG
     
     Equations 7-8 in Hurley et al 2000
     
     Parameters
     -----------
     an_coefficients :  array
     Array containing metallicity dependent an coefficients
     bn_coefficients :  array
     Array containing metallicity dependent bn coefficients
     Mass : def
     Mass in Msol
     coreMass : def
     Mass of core in Msol
     massCutoffs :  array
     Array containing metallicity dependent mass cutoffs
     logMetallicityXi : def
        log of metallicity
     
     Returns
     --------
     R_EHG : def
     Radius of a star at the end of the Hertzsprung Gap (HG) in Rsol
     
     
     """
    
    # Declare Variables
    MFGB         = massCutoffs[2];
    
    if(Mass < MFGB):
        LBGB = luminosityBaseGiantBranch(an_coefficients, Mass, m_randomSeed);
        return radiusFirstGiantBranch(bn_coefficients, Mass, LBGB, logMetallicityXi, m_randomSeed);
    
    else:
        return radiusHeliumIgnition(bn_coefficients, Mass, coreMass, massCutoffs, logMetallicityXi, m_randomSeed);
    





def radiusHeliumIgnition( bn_coefficients[nrowsb],  Mass,  coreMass,  massCutoffs[nMassCutoffs],  logMetallicityXi,  m_randomSeed):
    """
     Calculate the radius of the star at Helium Ignition (HeI)
     
     Given by Equation 50 in Hurley et al 2000
     
     Parameters
     -----------
     bn_coefficients :  array
     Array containing metallicity dependent bn coefficients for this star
     Mass : def
     Mass in Msol
     massCutoffs :  array
     Array containing metallicity dependent mass cutoffs
     
     Returns
     --------
     RHeI : def
     Radius at Helium Ignition (HeI) in Rsol
     
     
     """
    
    #
    MFGB = massCutoffs[2];
    LHeI = luminosityHeliumIgnition(bn_coefficients, Mass, massCutoffs, m_randomSeed);
    RmHe = minimumRadiusCoreHeliumBurning(bn_coefficients, Mass, coreMass, massCutoffs, logMetallicityXi, m_randomSeed);
    
    RAGB_LHeI = radiusAsymptoticGiantBranch(bn_coefficients, Mass, LHeI, massCutoffs, m_randomSeed);
    RGB_LHeI  = radiusFirstGiantBranch(bn_coefficients, Mass, LHeI, logMetallicityXi, m_randomSeed);
    
    if(Mass <= MFGB):
        return RGB_LHeI;
    
    elif(Mass >= std::max(MFGB, 12.0)):
        # DEBUGGING Check that for lower masses ~16 RmHe < RAGB_LHeI
        #print( << "RHeI = min(RmHe, RAGB_LHeI) = min(" << RmHe << "," << RAGB_LHeI << ")" << std::endl;
        return min(RmHe, RAGB_LHeI); # Given by Equation 55
    
    elif(Mass > MFGB and Mass < 12.0):
        muTop    = log10(Mass/12.0);
        muBottom = log10(MFGB / 12.0);
        mu       = muTop / muBottom;
        RHeI     = RmHe * pow((RGB_LHeI/RmHe), (mu));
        return RHeI;
    
    else:
        std::cerr << m_randomSeed << "\tError calculating radius at Helium Ignition (HeI)" << std::endl;
        return 0;
    
    





def radiusFirstGiantBranch(bn_coefficients[nrowsb], Mass,  Luminosity,  logMetallicityXi,  m_randomSeed):
    """
     Calculate the radius of a star on the first giant branch (FGB) as a function
     of the initial mass and the current core mass
     
     Given by Equation 46 in Hurley et al 2000
     
     Parameters
     -----------
     bn_coefficients :  array
     Array containing metallicity dependent bn coefficients for this star
     Mass : def
     Mass in Msol
     Luminosity : def
     Luminosity in Lsol
     
     Returns
     --------
     RGB : def
     Radius on the Giant Branch (GB) in Rsol
     
     
     """
#     A  = calculateRadiusGiantBranchConstantA(bn_coefficients, Mass);
    x           = calculateRadiusGiantBranchConstantX(logMetallicityXi, m_randomSeed);               # I think this can be precomputed and stored somewhere
    A           = calculateRadiusGiantBranchConstantAFromX(bn_coefficients, x, Mass);
    RGB         = A * (pow(Luminosity, bn_coefficients[1 - 1]) + bn_coefficients[2 - 1] * pow(Luminosity, bn_coefficients[3 - 1]));
    debugging   = false;
    

    
    return RGB;
    


SyntaxError: invalid syntax (<ipython-input-9-9a6544df88bc>, line 1)